In [128]:
%matplotlib inline

import os
import sqlite3
import hashlib
import random
import pickle
import numpy as np

from PIL import Image
from scipy.misc import imread
from matplotlib import pyplot as plt

In [95]:
_SQL_FACE_DATA = r"""
SELECT
  f.file_id AS file_id,
  fr.x as x,
  fr.y as y,
  fr.w as w,
  fr.h as h
FROM
  facerect AS fr
  JOIN faces AS f ON fr.face_id = f.face_id
WHERE
  fr.annot_type_id = 1
"""


def _read_face_data(aflw_sqlite_file):
    with sqlite3.connect(aflw_sqlite_file) as conn:
        c = conn.cursor()
        c.execute(_SQL_FACE_DATA)
        data = c.fetchall()
        face_data = {}
        for file_id, x, y, w, h in data:
            face_data.setdefault(file_id, []).append((x, y, w, h))
        # face_data has the format [(file_id, [(x, y, w, h), ...]), ...]
        return face_data.items()


def _crop_image(im, rect):
    x, y, w, h = rect
    return im[y:y+h, x:x+w]


def _scale_image(im, new_w, new_h):
    im = Image.fromarray(im).resize([new_w, new_h], Image.ANTIALIAS)
    return np.asarray(im)

def _rgb_to_gray(im):
    return np.dot(im / 255.0, [0.299, 0.587, 0.114])

def _gen_face_examples(im, face_rects, scale_to_w, scale_to_h):
    ans = []
    for r in face_rects:
        ex = _scale_image(_crop_image(im, r), scale_to_w, scale_to_h)
        if ex.shape == (scale_to_h, scale_to_w, 3):
            ans.append(ex)
    return ans

In [149]:
face_data = _read_face_data("../aflw/aflw/data/aflw.sqlite")
random.shuffle(face_data)

im_list = []
for i in range(len(face_data)):
    try:
        file_id, face_rects = face_data[i]
        im = imread(os.path.join("../aflw/aflw/data/flickr", file_id))
        im = _scale_image(_crop_image(im, face_rects[0]), 28, 28)
        im = _rgb_to_gray(im)
        im_list.append(im)
    except:
        pass

with open("face_data/all.pickle", "wb") as f:
    pickle.dump(im_list, f)